Get Train Predictions

In [ ]:

# import pandas as pd 
# import pickle
# from datetime import date, datetime

# ticker = 'adbe'

# ftraindf = pd.read_csv(f'{ticker}/data/ftraindf.csv', index_col = 'Date')
# ftraindf = ftraindf.reset_index(drop = False)
# ftraindf['Date'] = ftraindf.apply(lambda x: datetime.strptime(x.Date, '%Y-%m-%d').date(), axis = 1)
# ftraindf = ftraindf[ftraindf['Date'] >= date(2013, 1, 1)]
# ftraindf = ftraindf.reset_index(drop = True) 


# consol_df = pd.DataFrame() 

# for plus_target in range(1, 14, 1): 
    
#     resultant = ftraindf.copy()[['Date', 'yref_Tm0_close', 'refPrice_Tm14']]
    
#     df = pd.read_csv(f'{ticker}/data/train/Tp{plus_target}_train_clustermembership.csv', index_col = 'Date')
#     df = df.reset_index(drop = False)
#     df['Date'] = df.apply(lambda x: datetime.strptime(x.Date, '%Y-%m-%d').date(), axis = 1)
#     df = df[df['Date'] >= date(2013, 1, 1)]
#     df = df.reset_index(drop = True) 
    
#     resultant = pd.concat([resultant, df['pc_pred']], axis = 1)
#     resultant[f'Tp{plus_target}_pred'] = resultant.apply(lambda x: x[f'refPrice_Tm14'] * (1 + x['pc_pred']), axis = 1)
    
#     if len(consol_df) == 0: 
#         consol_df = resultant.copy()[['Date', 'yref_Tm0_close', f'Tp{plus_target}_pred']]
    
#     else: 
#         consol_df = pd.concat([consol_df, resultant[f'Tp{plus_target}_pred']], axis = 1)

# consol_df = consol_df.rename(columns = {'yref_Tm0_close': 'close'})
# consol_df = consol_df.set_index('Date')
# consol_df.to_csv(f'{ticker}/data/train/OVERALL_prediction_by_model.csv')
# consol_df

Get Reconstructed Data

In [ ]:

import pandas as pd 
import pickle
from datetime import date, datetime

ticker = 'xom'

train = pd.read_csv(f'{ticker}/data/train/OVERALL_prediction_by_model.csv')
train['Date'] = train.apply(lambda x: datetime.strptime(x.Date, '%Y-%m-%d').date(), axis = 1)
train = train.set_index('Date')

val = pd.read_csv(f'{ticker}/data/val/OVERALL_prediction_by_model.csv')
val['Date'] = val.apply(lambda x: datetime.strptime(x.Date, '%Y-%m-%d').date(), axis = 1)
val = val.set_index('Date')

test = pd.read_csv(f'{ticker}/data/test/OVERALL_prediction_by_model.csv')
test['Date'] = test.apply(lambda x: datetime.strptime(x.Date, '%Y-%m-%d').date(), axis = 1)
test = test.set_index('Date')

dataframe = pd.concat([train, val, test], axis = 0)
dataframe = dataframe[[col for col in dataframe.columns if 'date' not in col and 'close' not in col]]
dataframe

Prepare for historical data

In [ ]:

import pandas_ta as ta 

df = pd.DataFrame() 
df = df.ta.ticker(ticker)

df = df.reset_index(drop = False)
df['Date'] = df.apply(lambda x: x.Date.date(), axis = 1)
df = df.set_index('Date')

df1 = df.copy()[[]]
df1['Date1'] = df1.index
for index in range(60, 0, -1): 
    df1[f'Tm{index}_Price'] = df['Close'].shift(index)
df1 = pd.concat([df1, df['Close']], axis = 1)
df1 = df1[df1['Date1'] >= date(2013, 1, 1)][df1['Date1'] < date(2024, 1, 1)]

forecasted = pd.concat([df1, dataframe], axis = 1)
forecasted.to_csv(f'portfolio_data/vanilla/{ticker}_concatdata.csv')
forecasted

In [ ]:
def technical_indicators(x, st, lt, vanila = 0, lag = 0): 

    # initialize lag 
    if vanila == 1: 
        lag = 0
        rsi_lag = 0 
    else: 
        rsi_lag = 7 # to achieve the usage of half predicted and half historical 

    
    # calculate ema 26
    length = lt    
    ema26 = x[-(length+lag):]
    ema26 = ta.ema(ema26, length = length, adjust = False, min_periods = 1)
    ema26 = ema26.values[-(1+lag):][0]

    # calculate ema12 
    length = st
    ema12 = x[-(length+lag):]
    ema12 = ta.ema(ema12, length = length, adjust = False, min_periods = 1)
    ema12 = ema12.values[-(1+lag):][0]    
    
    # calculate sma 
    length = 26
    sma26 = x[-(length+lag):]
    sma26 = ta.sma(sma26, length = 26, adjust = False, min_periods = 1)
    sma26 = sma26.values[-(1+lag):][0]
    
    # calculate sma 
    length = 52
    sma52 = x[-(length+lag):]
    sma52 = ta.sma(sma52, length = length, adjust = False, min_periods = 1)
    sma52 = sma52.values[-(1+lag):][0]    
       
    
    # calculate macd 
    macd = ema12 - ema26
    
    # calculate rsi 
    length = 15
    rsi = x[-(length+rsi_lag):]
    rsi = ta.rsi(rsi, length = 14)
    rsi = rsi.values[-(1+rsi_lag):][0]
    
    return (macd, rsi, sma26, sma52)

In [ ]:
def experiment1(item, candidate, vanila = 0, lag = 0, buy_threshold = 0, sell_threshold = 0, choice_action = 'macd'):
    
    short = candidate['short']
    long = candidate['long']
    signal = candidate['signal']
    buy_threshold = candidate['buy_threshold']
    sell_threshold = candidate['sell_threshold']
    
      
    # forecasted values (current day)
    results = item.copy()
    
    # obtain ema & rsi values 
    results[['macd_1', 'rsi_1', 'sma26_1', 'sma_52']] = results.apply(lambda x: technical_indicators(x, st = short, lt = long, vanila = vanila, lag = lag), axis = 1, result_type='expand')
    # ema of MACD 
    results['macd_signal_1'] = ta.ema(results['macd_1'], length = signal, adjust = False, min_periods = 1)
    # MACD Histogram 
    results['macd_div_1'] = results['macd_1'] - results['macd_signal_1']
   
    return results

In [ ]:

targetData = forecasted.copy()[[col for col in forecasted.columns if 'pred' not in col]]

print(targetData.columns)

candidate = {
    'short': 12, 
    'long': 26, 
    'signal': 9, 
    'buy_threshold': 0, 
    'sell_threshold': 0, 
}

results = experiment1(targetData, candidate, vanila = 0, lag = 0)
results

In [ ]:
results_out = results.copy()[[col for col in results.columns if 'macd_1' in col or 'sma' in col]]
results_out

In [ ]:
results_out.to_csv(f'portfolio_data/vanilla/{ticker}_portdata.csv')